# **Übung 8** Programmierung mit Python mit Anwendungen aus dem Maschinellen Lernen



[Einführung Neuronale Netze](https://playground.tensorflow.org/)
- Data Pattern (linear, nicht-linear trennbar)
- Datenset (Noise, Train/Test-Split)
- Epoche
- Aktivierungsfunktionen (Linear, TanH)
- Auswirkungen der Learning Rate
- Design Netzwerk (Größe, Breite)
- Overfitting
- Stabilität des Trainings

## Aufgabe 1
Die erste Aufgabe beschäftigt sich mit der Lineare Regression in PyTorch.

In [1]:
# Hilfsfunktionen

import plotly.express as px
import plotly.graph_objects as go
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

SAMPLES_PER_AXIS = 100

def _generate_dataset(n, m, t, s, e, with_noise=True):
    x1 = np.tile(np.linspace(s, e, SAMPLES_PER_AXIS), SAMPLES_PER_AXIS)
    x2 = np.repeat(np.linspace(s, e, SAMPLES_PER_AXIS), SAMPLES_PER_AXIS)
    r = np.sin(x1)*np.sin(x2)*3 + (np.random.random(SAMPLES_PER_AXIS*SAMPLES_PER_AXIS)-0.5)*15 if with_noise else np.zeros_like(x1)
    y = n*x1 + m*x2 + t + r

    return pd.DataFrame(np.vstack([x1, x2, y, r]).T, columns=['x1', 'x2', 'y', 'r'])


def generate_dataset(n, m, t):
    ''' f(x1,x1) = n*x1 + m*x2 + t'''

    train = _generate_dataset(n, m, t, 0, 50)
    test = _generate_dataset(n, m, t, 50, 200, with_noise=False)

    return train, test


def show_dataset(dl, title):
    SAMPLES_PER_AXIS1 = SAMPLES_PER_AXIS* int(len(dl) / (SAMPLES_PER_AXIS*SAMPLES_PER_AXIS))
    rs = lambda x: x.values.reshape(SAMPLES_PER_AXIS1,SAMPLES_PER_AXIS)

    data = [go.Surface(z=rs(dl['y']),x=rs(dl['x1']),y=rs(dl['x2']), surfacecolor=rs(dl['r'])) for ds in dl]
    fig = go.Figure(data=data)

    fig.update_layout(
        title=title, 
        autosize=True, 
        )
    
    fig.update_scenes(xaxis_title_text='x1',  
                  yaxis_title_text='x2',  
                  zaxis_title_text='y')

    fig.show()

**Aufgabe 1.1 (PyTorch)** | Gegeben ist ein Datensatz mit zwei Input-Parameters $x1$ und $x2$ und einem gemessenen Output-Parameter $y$
Es ist bekannt, dass sich $y$ durch folgende Funktion bestimmt ist: $f(x1,x2) = nx1 + mx2 + t + r(x1,x2)$

$r(x1,x2)$ überlagert die lineare Funktion mit einem gleichverteilten, periodischen Rauschen.

Mithilfe einer Linearen Regression sollen für unbekannte $x1, x2$ paare $y$-Werte abgeschätzt werden. Im Folgenden soll eine Lineare Regression mit PyTorch durchgeführt werden. Sie können sich an den Inhalten im Videomaterial zu dieser Übung orientieren.

Schauen Sie sich den gegebenen Datensatz an, der Code hierfür ist schon vorgegeben.


In [2]:
generated_train_df, generated_test_df = generate_dataset(n=2, m=-1, t=60)
show_dataset(generated_train_df, title='[Train] Noisy Linear Surface') 
show_dataset(generated_test_df, title='[Test] Flat Linear Surface')
show_dataset(pd.concat([generated_train_df, generated_test_df], ignore_index=True), title='[Train+Test] Combined') 


label_train_df = generated_train_df.pop("y")
generated_train_df.pop("r")
label_test_df = generated_test_df.pop("y")
_ = generated_test_df.pop("r")



Output hidden; open in https://colab.research.google.com to view.

**Aufgabe 1.2 (PyTorch)** | Zunächst soll der Datensatz in Tensoren konvertiert werden.
Erzeugen Sie sich die vier Variablen `X_train, y_train, X_test, y_test`

```
Erwartete Ausgabe der gegebenen Prints:
X_train torch.Size([10000, 2]) torch.float32 cpu
y_train torch.Size([10000, 1]) torch.float32 cpu
X_test  torch.Size([10000, 2]) torch.float32 cpu
y_test  torch.Size([10000, 1]) torch.float32 cpu
```

In [3]:
import torch
import torch.nn as nn
import numpy as np
import plotly.express as px

# Daten in Tensoren konvertieren
def create_tensor(featues, label):
    X_data = torch.from_numpy(featues.values.astype(np.float32))
    y_data = torch.from_numpy(label.values.astype(np.float32))
    y_data = y_data.view(y_data.shape[0], 1)
    return X_data, y_data

X_train, y_train = create_tensor(generated_train_df, label_train_df)
X_test, y_test = create_tensor(generated_test_df, label_test_df)

print("X_train", X_train.shape, X_train.dtype, X_train.device)
print("y_train", y_train.shape, y_train.dtype, y_train.device)
print("X_test ", X_test.shape, X_test.dtype, X_test.device)
print("y_test ", y_test.shape, y_test.dtype, y_test.device)




X_train torch.Size([10000, 2]) torch.float32 cpu
y_train torch.Size([10000, 1]) torch.float32 cpu
X_test  torch.Size([10000, 2]) torch.float32 cpu
y_test  torch.Size([10000, 1]) torch.float32 cpu


**Aufgabe 1.3 (PyTorch)** | Erzeugen Sie ein Modell, welches die Parameter $n,m,t$ als trainierbare Parameter enthält. Sie können hierfür die Funktion selbst modellieren oder das `torch.nn.Linear`-Modul verwenden.

Weisen Sie das Modell der Variablen `model` zu.



In [4]:
model = nn.Linear(2, 1)

**Aufgabe 1.4 (PyTorch)** | Als nächstes muss eine geeignete Loss-Funktion für dieses Problem ausgewählt werden. Die Loss-Funktion ist ein Maß für den Grad der Abweichung zwischen den vorhergesagten Ausgaben des Modells und den tatsächlichen Ausgaben.

Die Distanz, zwischen vorhergesagten und tatsächlichen Wert, könnte hier ein gutes Maß sein. Im Namespace `torch.nn` sind verschiedene Loss-Funktionen definiert. Wählen Sie eine aus, oder implementieren Sie diese selbst.

Weisen Sie die Loss-Funktion der Variablen `loss_fn` zu. 

In [5]:
loss_fn = nn.MSELoss()

**Aufgabe 1.5 (PyTorch)** | Als letztes muss ein Optimizer erstellt werden. Ein Optimizer ist ein Algorithmus, der verwendet wird, um die Parameter eines Modells auf der Grundlage der Ergebnisse der Loss-Funktion zu optimieren. Erstellen Sie eine Instanz von `torch.optim.SGD` (stochastic gradient descent). Diesem müssen Sie die Modellparameter `model.parameters()` und eine `learning_rate` übergeben. 

Die `learning_rate` gibt an, wie stark die Parameter durch die Gradienten beeinflusst werden. Hier gilt es, ein richtiges Maß zu finden.

In [6]:
learning_rate = 0.0006
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Aufgabe 1.6 (PyTorch)** | Trainieren Sie das erstellte Modell. In der Regel wird das Modell pro Epoche einmal mit allen Trainingsdaten trainiert. Folgende Schritte sollen in jeder Epoche durchgeführt werden.
1. Modellausgaben `y_pred_train`, mit Trainingsdaten `X_train`, erzeugen
2. Berechnen des Loss `loss_train` durch Vergleich von `y_pred_train` und `y_train`
3. Den Gradienten auf Basis des Loss berechnen
4. Modellparameter optimieren
5. Modellausgaben `y_pred_test`, mit Testdaten `X_test`, erzeugen
6. Berechnen des Loss `loss_test` durch Vergleich von `y_pred_test` und `y_test`
7. Gradienten zurücksetzen


Vergleichen Sie die Modellparameter $n,m,t$


In [7]:
learning_rate = 0.0006
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_res = []

num_epochs = 25000
plot_freq = 10
print_freq = 100

for epoch in range(num_epochs):
  y_pred_train = model(X_train)
  loss_train = loss_fn(y_pred_train, y_train)

  loss_train.backward()

  optimizer.step()

  y_pred_test = model(X_test)
  loss_test = loss_fn(y_pred_test, y_test)

  optimizer.zero_grad()

  if (epoch+1) % plot_freq == 0:
    loss_res.append([epoch, loss_train.item(), loss_test.item()])

  if (epoch+1) % print_freq == 0:
    print(f'epoch: {epoch+1}, loss_train = {loss_train.item():.4f}, loss_test = {loss_test.item():.4f}')

df = pd.DataFrame(np.array(loss_res), columns=['epoch', 'loss_test', 'loss_train'])

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

fig = px.line(df, x="epoch", y=['loss_test', 'loss_train'], title='Trainingsverlauf', log_y=True)
fig.show()



epoch: 100, loss_train = 525.9278, loss_test = 41300.4570
epoch: 200, loss_train = 508.6167, loss_test = 39888.2695
epoch: 300, loss_train = 491.8988, loss_test = 38524.3828
epoch: 400, loss_train = 475.7534, loss_test = 37207.2031
epoch: 500, loss_train = 460.1613, loss_test = 35935.1094
epoch: 600, loss_train = 445.1034, loss_test = 34706.5352
epoch: 700, loss_train = 430.5613, loss_test = 33520.0195
epoch: 800, loss_train = 416.5175, loss_test = 32374.1191
epoch: 900, loss_train = 402.9548, loss_test = 31267.4141
epoch: 1000, loss_train = 389.8568, loss_test = 30198.5820
epoch: 1100, loss_train = 377.2074, loss_test = 29166.3457
epoch: 1200, loss_train = 364.9915, loss_test = 28169.4277
epoch: 1300, loss_train = 353.1940, loss_test = 27206.6113
epoch: 1400, loss_train = 341.8008, loss_test = 26276.7617
epoch: 1500, loss_train = 330.7979, loss_test = 25378.7109
epoch: 1600, loss_train = 320.1719, loss_test = 24511.4062
epoch: 1700, loss_train = 309.9100, loss_test = 23673.7656
epoch:

**Aufgabe 1.7 (PyTorch)** | Abschließend sollen Sie versuchen, das obige Beispiel durch einfache Veränderungen deutlich schneller Trainieren zu.

Dazu könnten folgende verändert werden.
- `learning_rate` anpassen
- anderen Optimierer testen, z.B. Adam
- Input normieren
- Verändern Sie auch den Parameter $t$

In [ ]:

# Normierung der Inputdaten → Training läuft deutlich schneller ab. SGD ist auch bei großen 't' werten stabil
# Beispiel: 
m,s = X_train.mean(), X_train.std()
X_train = (X_train - m) / s
X_test = (X_test - m) / s

# Adam, trainiert auch bei großen 't' werten stabil
# In diesem Fall kann bei Adam die lerning_rate sehr hoch gesetzt werden, ohne dass das Training instabil verläuft.